In [1]:
from augment import *
import glob
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                               #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from GAN import *                  
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #

In [2]:
def dstnc(dt2,dt1): #فاصله و واریانس بردار فاصله ی هر کدام از ردیف های دیتاست(آگمنت)1 از (کل)2
    dist,vrnc,i,h,w=[],[],0,np.shape(dt1)[0],np.shape(dt1)[1]
    end=min(len(dt2[0]),len(dt1[0]))
    for i in range (h):
        ds1 = dt2-dt1[i,:end] #.reshape(-1,1) #distance of every training instance
        dist_array=(ds1*10)**3  #معیار فاصله با حساسیت به توان 3 (حفظ علامت و تاثیر بیشتر تفاوت زیاد در نقاط)
        vr=np.var(ds1)
        ds1=min(np.sum(np.absolute(dist_array),axis=1))
        #print('dist2 = ', dist2)
        vrnc.append(vr)
        dist.append(ds1)
    variance=np.reshape(vrnc,(-1,1))
    distnc=np.reshape(dist,(-1,1))
    return(distnc,variance)

cls_num=2
ecg=np.array([])
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Ham(i)         #فراخوانی داده های اصلی
    ecg=np.append(ecg,vars()['ecg'+str(i)])
wdth=int(len(ecg1[0]))
ecg=np.reshape(ecg,(int(len(ecg)/wdth),wdth))  #base data windows
print('\nfinal_train_shap=',np.shape(ecg))
os.chdir(cwd)

first_train_shape= (109, 432)
first_test_shape= (105, 432)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [52 57]
max(train_feature_Altitude)= 8.0329
min(train_feature_Altitude)= -2.0538
first_train_sample=
 [ 1.         0.11403    0.10489    0.095754   0.086619   0.086619
  0.086619   0.077484   0.077484   0.077484   0.077484   0.077484
  0.077484   0.077484   0.077484   0.077484   0.077484   0.077484
  0.068348   0.068348   0.068348   0.068348   0.068348   0.068348
  0.068348   0.068348   0.068348   0.077484   0.077484   0.077484
  0.077484   0.086619   0.086619   0.095754   0.095754   0.10489
  0.11403    0.11403    0.12316    0.12316    0.1323     0.14143
  0.14143    0.15057    0.1597     0.1597     0.16884    0.17797
  0.19624    0.20538    0.22365    0.25106    0.30587    0.36068
  0.37895    0.37895    0.43376    0.49771    0.47031    0.45204
  0.70783    1.2742     1.5026     1.4843     1.3564     1.2742
  1.192      1.0184     0.91794    0.89054    0.85399    0.70783
  0.

In [3]:
base_wn=np.array([])
windws=np.array([])

for cls in range (1,cls_num+1):                   #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])
    base_wn=np.append(base_wn,vars()['wndws'+str(cls)])
base_wn=np.reshape(base_wn,(int(len(base_wn)/wdth),wdth))  #base data windows
print("shape wndws1=", np.shape(wndws1))
print("shape base_wn=", np.shape(base_wn))

furir_windws0=np.fft.fft(np.array(base_wn[:,:-1]))
print("shape furir_windws0=", np.shape(furir_windws0))
furir_windws=np.concatenate((np.array(furir_windws0),np.transpose([base_wn[:,-1]])),axis=1)   
print("shape furir_windws=", np.shape(furir_windws))

smpl_rte=len(base_wn[0])-1                 #len(ecg1[0])  # در ماژول ها نیز همین مقدار ثبت شده
print("smpl_rte=",smpl_rte)
mx_wndws=np.max(np.bincount(np.int16(base_wn[:,-1])))
print("mx_wndws=",mx_wndws)

shape wndws1= (52, 481)
shape base_wn= (109, 481)
shape furir_windws0= (109, 480)
shape furir_windws= (109, 481)
smpl_rte= 480
mx_wndws= 57


In [4]:
path=('{}\AugEvl'.format(cwd)) #create folder 
if not os.path.isdir(path):
    os.mkdir(path)

def addqlty(base_wn_dt,data,mthd,i):               
    cls=int(i)
    c,alpha,PL=0,1,1
    #from glob import glob
    cwd = os.getcwd() #
    adrs=cwd+'/Aug/aug'+str(i)+str(mthd)+'.csv'
    #alldata = np.array([])                    # List for storing all the data
    train_labels=np.array(base_wn_dt[:,-1])
    tr=np.array(base_wn_dt[:,:-1])
    indx_pstv=np.where(base_wn_dt[:,-1] == i)
    P=np.array(base_wn_dt[indx_pstv])
    indx_ngtv=np.where(base_wn_dt[:,-1] != i)
    N=np.array(base_wn_dt[indx_ngtv])

    #data = np.array(data)
    aug=np.array(data)
    pnts=int(np.shape(data)[1])
    
    #aug=data[data[:,-2].argsort()]        # Positive data Sorted Base Distance
    trl=np.array(train_labels)
    for j in range (len(trl)):
        if train_labels[j]==i :           # !!!!! == (Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    #mn_ds_pstv=np.reshape(aug[:,-2],(-1,1)) #min dis to positive class (submited in -2 column)
    print('shape and aug[0]=',np.shape(aug[0]),aug[0])
    print('shape P and P[0]=',np.shape(P),P[0])    
    qlty1,qlty2=mn_ds_pstv,vr_P=dstnc(P[:,1:-1],aug[:,1:-1])  #min distance to negative class
    new1=np.append(aug,qlty1,axis=1)
    new2=np.append(new1,qlty2,axis=1)
    mn_ds_ngtv,vr_N=dstnc(N[:,1:-1],aug[:,1:-1])              #min distance to negative class
    qlty3=mn_ds_pstv/mn_ds_ngtv                               #Smaller is Better
    qlty4=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new3=np.append(new2,qlty3,axis=1)
    new4=np.append(new3,qlty4,axis=1)
    for j in range (len(trl)):
        if train_labels[j]!=i :                          # !!!!! == (!= Inverse Paper GDO)
            trl[j]=1
        else :
            trl[j]=0
    trlt=np.array([trl]).T
    qlty5=np.reshape(valuation(aug[:,:-1],tr,trlt),(-1,1))   #line 272
    new5=np.append(new4,qlty5,axis=1)
    print('class=',i,'shape(new5)=',np.shape(new5),end='\t')
    print('mx=',np.max(new5[:,1:-7]),end='\t')
    print('mn=',np.min(new5[:,1:-7]))
    pd.DataFrame(new5[:,:]).to_csv('{}/AugEvl/AugEvl{}{}.csv'.format(cwd,cls,mthd))
    print()

In [5]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug'+str(i)+'00']=np.empty((0,smpl_rte+1), float) #Scale    
    vars()['ecg_aug'+str(i)+'10']=np.empty((0,smpl_rte+1), float) #Scale
    vars()['ecg_aug'+str(i)+'20']=np.empty((0,smpl_rte+1), float) #Mag Warp
    vars()['ecg_aug'+str(i)+'30']=np.empty((0,smpl_rte+1), float) #TimWarp
    vars()['ecg_aug'+str(i)+'40']=np.empty((0,smpl_rte+1), float) #GDO
    vars()['ecg_aug'+str(i)+'50']=np.empty((0,smpl_rte+1), float) #frequence
    vars()['ecg_aug'+str(i)+'60']=np.empty((0,smpl_rte+1), float) #GAN
    vars()['ecg_aug'+str(i)+'61']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'62']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'70']=np.empty((0,smpl_rte+1), float) #LSTM
    vars()['ecg_aug'+str(i)+'71']=np.empty((0,smpl_rte+1), float)
    vars()['ecg_aug'+str(i)+'72']=np.empty((0,smpl_rte+1), float)

In [6]:
for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vlum_sig=10*mx_wndws-len(vars()['wndws'+str(i)])
    
    vars()['ecg_aug'+str(i)+'10']=np.append(vars()['ecg_aug'+str(i)+'10'],scl_rndm_beat(ecg,i,int(vlum_sig),mag=.1),axis=0)#0.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'10'],10,i)
    print(str(i)+'10 =',len(vars()['ecg_aug'+str(i)+'10']))
    
    vars()['ecg_aug'+str(i)+'20']=np.append(vars()['ecg_aug'+str(i)+'20'],MgWrp_rndm_beat(ecg,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'20'],20,i)
    print(str(i)+'20 =',len(vars()['ecg_aug'+str(i)+'20']))
    
    vars()['ecg_aug'+str(i)+'30']=np.append(vars()['ecg_aug'+str(i)+'30'],TimWrpRaw(ecg,i,int(vlum_sig),mag=.1),axis=0)#.1
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'30'],30,i)
    print(str(i)+'30 =',len(vars()['ecg_aug'+str(i)+'30']))
    
    vars()['ecg_aug'+str(i)+'40']=np.append(vars()['ecg_aug'+str(i)+'40'],GDORaw(ecg,i,int(vlum_sig),mag=.05),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'40'],40,i)
    print(str(i)+'40 =',len(vars()['ecg_aug'+str(i)+'40']))
    
    vars()['ecg_aug'+str(i)+'50']=np.append(vars()['ecg_aug'+str(i)+'50'],frqncRaw(furir_windws,i,int(vlum_sig),mag=.2),axis=0)
    addqlty(base_wn,vars()['ecg_aug'+str(i)+'50'],50,i)
    print(str(i)+'50 =',len(vars()['ecg_aug'+str(i)+'50']))

shape and aug[0]= (481,) [-8.53423963e-01 -8.60854187e-01 -8.44124912e-01 -8.61387526e-01
 -8.41003802e-01 -8.46922168e-01 -8.23688458e-01 -8.28915812e-01
 -8.22865740e-01 -8.32218425e-01 -8.39033653e-01 -8.41726607e-01
 -8.47890306e-01 -8.49989043e-01 -8.72007694e-01 -8.92178311e-01
 -9.10630351e-01 -9.09066884e-01 -9.06470751e-01 -8.88434894e-01
 -8.74856265e-01 -8.72986301e-01 -8.72162654e-01 -8.75262070e-01
 -8.68407733e-01 -8.41587143e-01 -7.47969535e-01 -5.80143926e-01
 -4.07728767e-01 -3.32126449e-01 -3.34629084e-01 -4.28877774e-01
 -5.99009899e-01 -7.17366873e-01 -7.66723935e-01 -7.68261888e-01
 -7.55919233e-01 -7.61943603e-01 -7.95401227e-01 -8.46048517e-01
 -8.91016124e-01 -9.09712726e-01 -9.06653325e-01 -8.89887939e-01
 -8.50831326e-01 -7.92187859e-01 -7.20228655e-01 -6.65890532e-01
 -6.37005941e-01 -6.24424056e-01 -6.14099155e-01 -5.95187270e-01
 -5.61137482e-01 -4.99148203e-01 -4.28175827e-01 -3.60456989e-01
 -2.82571250e-01 -1.75587230e-01 -3.03043439e-02  9.12153962e-02


class= 1 shape(new5)= (518, 486)	mx= 9.42165852686803	mn= -2.2835317385746388

110 = 518
shape and aug[0]= (481,) [-6.27610447e-01 -5.74253740e-01 -5.05868383e-01 -4.98854527e-01
 -5.26427087e-01 -5.61105223e-01 -5.80343721e-01 -5.69602516e-01
 -5.25481643e-01 -4.69552609e-01 -4.36904878e-01 -4.25935258e-01
 -4.18985751e-01 -4.16733779e-01 -4.18888083e-01 -4.39616269e-01
 -4.82881186e-01 -5.39985549e-01 -5.92476720e-01 -6.42695909e-01
 -6.82885453e-01 -6.98869449e-01 -6.98939949e-01 -6.67995189e-01
 -6.52200263e-01 -6.70776996e-01 -6.88548733e-01 -7.07053656e-01
 -7.05009336e-01 -6.90748507e-01 -6.24182424e-01 -5.07407184e-01
 -4.57745105e-01 -4.77508347e-01 -6.02876839e-01 -7.73144080e-01
 -8.47211231e-01 -8.68176510e-01 -8.65145221e-01 -8.81909101e-01
 -9.37294343e-01 -1.00773918e+00 -1.07187294e+00 -1.10997864e+00
 -1.11628345e+00 -1.09485966e+00 -1.05886880e+00 -9.90441666e-01
 -9.35546408e-01 -8.92034485e-01 -8.52738453e-01 -8.15754270e-01
 -7.67410603e-01 -6.98513387e-01 -5.73704

class= 1 shape(new5)= (520, 486)	mx= 10.60647730910777	mn= -2.5383002843518074

120 = 520


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


shape and aug[0]= (481,) [-5.80270000e-01 -5.81752531e-01 -5.83049277e-01 -5.77296514e-01
 -5.72210534e-01 -5.74893757e-01 -5.76109194e-01 -5.69610776e-01
 -5.65541485e-01 -5.68109946e-01 -5.68229549e-01 -5.64720933e-01
 -5.64430768e-01 -5.66620574e-01 -5.66253963e-01 -5.65049590e-01
 -5.66291758e-01 -5.67601187e-01 -5.68857765e-01 -5.70621039e-01
 -5.72943592e-01 -5.76167859e-01 -5.79507887e-01 -5.81163534e-01
 -5.82840412e-01 -5.84545636e-01 -5.84331282e-01 -5.83847601e-01
 -5.84319596e-01 -5.83508609e-01 -5.81852480e-01 -5.79988666e-01
 -5.77670130e-01 -5.74920202e-01 -5.72716606e-01 -5.70714146e-01
 -5.68909536e-01 -5.67257473e-01 -5.65817157e-01 -5.64413600e-01
 -5.62596765e-01 -5.60337389e-01 -5.57375583e-01 -5.53432627e-01
 -5.47223175e-01 -5.37600089e-01 -5.23792166e-01 -5.04816028e-01
 -4.78653390e-01 -4.45979704e-01 -4.11959898e-01 -3.81464143e-01
 -3.58651368e-01 -3.45063133e-01 -3.41806735e-01 -3.40951116e-01
 -3.48645062e-01 -3.74637350e-01 -4.14979920e-01 -4.55604371e-01


...

class= 2 shape(new5)= (513, 486)	mx= 9.574548249995603	mn= -1.7796361608267415

210 = 513
shape and aug[0]= (481,) [-1.01459433e+00 -1.01696852e+00 -9.77979407e-01 -9.71804214e-01
 -9.16377476e-01 -9.25116002e-01 -9.06339175e-01 -9.24291340e-01
 -9.23704722e-01 -9.26774410e-01 -9.19552272e-01 -9.03126102e-01
 -8.81601992e-01 -8.39086951e-01 -8.02528464e-01 -7.55116582e-01
 -7.36419656e-01 -7.23182031e-01 -7.26142893e-01 -7.43294516e-01
 -7.69386172e-01 -7.80866074e-01 -7.76334570e-01 -7.70130799e-01
 -7.42860751e-01 -7.24279009e-01 -7.11044658e-01 -7.12393472e-01
 -7.12572859e-01 -7.15773070e-01 -7.08260284e-01 -6.62485924e-01
 -5.73595336e-01 -4.75003587e-01 -4.36967745e-01 -4.20010270e-01
 -4.72898318e-01 -6.08502876e-01 -7.14618383e-01 -7.73094941e-01
 -7.89566268e-01 -7.83655376e-01 -7.62643476e-01 -7.37394085e-01
 -7.40399915e-01 -7.62436860e-01 -7.81191529e-01 -7.85297239e-01
 -7.84922519e-01 -7.81280445e-01 -7.69016917e-01 -7.44638199e-01
 -7.10281574e-01 -6.71675029e-01 -6.2030

class= 2 shape(new5)= (515, 486)	mx= 10.096782380846456	mn= -2.1217519561621203

220 = 515
shape and aug[0]= (481,) [-6.18570000e-01 -5.84514102e-01 -5.57267647e-01 -5.10880001e-01
 -4.91914084e-01 -4.80104802e-01 -4.65726820e-01 -4.62031645e-01
 -4.60399286e-01 -4.63714205e-01 -4.59234007e-01 -4.62475391e-01
 -4.61761490e-01 -4.66299018e-01 -4.77447754e-01 -4.86728919e-01
 -5.01886505e-01 -5.12691834e-01 -5.16473599e-01 -5.22693072e-01
 -5.26561538e-01 -5.31652366e-01 -5.32279052e-01 -5.32665717e-01
 -5.35410474e-01 -5.37882953e-01 -5.39201340e-01 -5.45245822e-01
 -5.48008723e-01 -5.55170731e-01 -5.61960748e-01 -5.67483528e-01
 -5.71725653e-01 -5.72696106e-01 -5.75519139e-01 -5.77841466e-01
 -5.77976873e-01 -5.78273376e-01 -5.73168909e-01 -5.71488935e-01
 -5.63068682e-01 -5.54289275e-01 -5.35229464e-01 -5.11545212e-01
 -4.78794276e-01 -4.35704788e-01 -3.83861293e-01 -3.20069620e-01
 -2.44481730e-01 -1.67005529e-01 -8.08791002e-02 -5.22514130e-03
  5.64822566e-02  9.91562342e-02  1.144

class= 2 shape(new5)= (516, 486)	mx= 7.785297823846159	mn= -1.5755095136332007

230 = 516
shape and aug[0]= (481,) [-6.66876930e-01 -6.97503084e-01 -7.36522005e-01 -7.40574029e-01
 -7.43850239e-01 -7.21198909e-01 -6.95961807e-01 -6.80797976e-01
 -6.82343460e-01 -6.87018802e-01 -6.96636249e-01 -7.03424983e-01
 -7.06412311e-01 -7.12317001e-01 -7.22027121e-01 -7.33392079e-01
 -7.36211315e-01 -7.31048009e-01 -7.00692895e-01 -6.46412479e-01
 -6.08924896e-01 -6.01938785e-01 -6.09155002e-01 -6.27507038e-01
 -6.36835649e-01 -6.32796013e-01 -6.01181242e-01 -5.51046978e-01
 -5.25327081e-01 -5.22748125e-01 -5.42699828e-01 -5.95736182e-01
 -6.38844790e-01 -6.48931721e-01 -6.37503374e-01 -5.86371798e-01
 -5.35901099e-01 -5.20837397e-01 -5.36641736e-01 -6.00523414e-01
 -6.69503292e-01 -7.09710906e-01 -7.25251402e-01 -7.15201608e-01
 -6.77100478e-01 -6.07681753e-01 -5.27317973e-01 -4.61093392e-01
 -4.30255599e-01 -4.23767166e-01 -4.21997759e-01 -4.33607212e-01
 -4.40678144e-01 -4.41145452e-01 -4.3750

class= 2 shape(new5)= (516, 486)	mx= 7.830409725405701	mn= -1.593444803611717

240 = 516
shape and aug[0]= (481,) [-0.98291016 -0.90429688 -0.76953125 -0.71972656 -0.69580078 -0.71582031
 -0.73291016 -0.79248047 -0.84667969 -0.90478516 -0.94970703 -0.98144531
 -1.00585938 -1.01464844 -1.02636719 -1.02539062 -1.03125    -1.03027344
 -1.03222656 -1.03027344 -1.02929688 -1.02734375 -1.01953125 -1.01074219
 -0.98974609 -0.97021484 -0.94433594 -0.92578125 -0.90869141 -0.89990234
 -0.89550781 -0.89306641 -0.89501953 -0.89550781 -0.89990234 -0.89697266
 -0.89111328 -0.87646484 -0.86279297 -0.84912109 -0.83740234 -0.82910156
 -0.82275391 -0.82177734 -0.82373047 -0.83496094 -0.84765625 -0.86376953
 -0.87988281 -0.89697266 -0.90673828 -0.90869141 -0.90673828 -0.88916016
 -0.85595703 -0.80712891 -0.75195312 -0.69335938 -0.64111328 -0.60253906
 -0.58544922 -0.58496094 -0.58984375 -0.60595703 -0.62988281 -0.65722656
 -0.68066406 -0.703125   -0.72119141 -0.73339844 -0.74169922 -0.74609375
 -0.749023

class= 2 shape(new5)= (516, 486)	mx= 7.80859375	mn= -1.59375

250 = 516
